In [0]:
dbutils.notebook.exit(1)

# FNB Gold – Business-Ready Aggregated Layer

This notebook transforms the curated Silver-layer FNB data into business-ready metrics.  
The Gold table is modeled for analytics dashboards, reporting, and downstream consumption.

Data Source:

| Tables |
| ------ |
| default.silver_fnb_sales |

Target Tables:

| Tables |
| ------ |
| default.gold_fnb_sales |


In [0]:
%sql
ALTER TABLE default.gold_fnb_sales SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name')

In [0]:
%sql
ALTER TABLE default.gold_fnb_sales
DROP COLUMN AVERAGE_UNIT_PRICE;

### Imports

In [0]:
from pyspark.sql.functions import sum, trunc, current_timestamp

### Read from Silver

In [0]:
df_silver = spark.read.table('default.silver_fnb_sales')

### Transformation

#### Monthly conversion

In [0]:
df_silver = df_silver.withColumn('DATE',trunc("DATE", "month"))

#### Aggregate based on dimensions

In [0]:
df_silver_agg = (
    df_silver
    .groupBy("COUNTRY", "RETAILER", "CATEGORY","SEGMENT", "BRAND", "MANUFACTURER","CURRENCY_CODE", "DATE")
    .agg(
        sum("SALES").alias("SALES"),
        sum("UNITS").alias("UNITS"),
        sum("VOLUME").alias("VOLUME"),
        # sum("AVERAGE_UNIT_PRICE").alias("AVERAGE_UNIT_PRICE"),
        sum("DISTRIBUTION").alias("DISTRIBUTION"),
        sum("PROMO_SALES").alias("PROMO_SALES"),
        sum("PROMO_UNITS").alias("PROMO_UNITS"),
        sum("SALES_INR").alias("SALES_INR"),
        sum("SALES_GBP").alias("SALES_GBP")
    )
)

In [0]:
df_gold = df_silver_agg.withColumn('LAST_UPDTAED',current_timestamp())

### Save as Table

In [0]:
df_gold.count()

In [0]:
# df_gold.write.format('delta').mode('overwrite').option("mergeSchema", "true").partitionBy('CATEGORY','RETAILER').saveAsTable('default.gold_fnb_sales')